In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [55]:
def initialize_bias(shape):
    initialize_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(initialize_bias_vals)

In [56]:
def initialize_weights(shape):
    initialize_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initialize_random_dist)

In [57]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [58]:
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [59]:
def convolutional_layer(input_x, shape):
    W = initialize_weights(shape)
    b = initialize_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [60]:
def fully_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [61]:
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])

In [62]:
input_image = tf.reshape(x,[-1,28,28,1])

In [64]:
convolutional_layer1 = convolutional_layer(input_image, shape =[4,4,1,32])
pooling_layer1 = max_pool(convolutional_layer1)
convolutional_layer2 = convolutional_layer(pooling_layer1, shape =[4,4,32,64])
pooling_layer2 = max_pool(convolutional_layer2)

In [65]:
convo_2_flat = tf.reshape(pooling_layer2,[-1,7*7*64])
full_layer_one = tf.nn.relu(fully_connected_layer(convo_2_flat,1024))

In [66]:
holding_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=holding_prob)

In [67]:
y_prediction =fully_connected_layer(full_one_dropout,10)

In [68]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_prediction))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [69]:
init = tf.global_variables_initializer()

In [71]:
step = 10
with tf.Session() as session:
    session.run(init)
    
    for i in range(step):
        input_x , input_y = mnist.train.next_batch(100)
        session.run(train,feed_dict={x:input_x,y:input_y,holding_prob:0.5})
    
        if i%5 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_prediction,1),tf.argmax(y,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(session.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels,holding_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1229


Currently on step 5
Accuracy is:
0.1611


